In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf

/work/modules/Ubuntu/14.04/amd64/common/anaconda2/latest/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Path to the source code folder that includes 'iris_data.py'
import sys
sys.path.append('~/tensorflow/models/samples/core/get_started/')
import iris_data

In [3]:
# Pretend the argv and input arguments to make it return to the default values
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

args = parser.parse_args([])

In [18]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [26]:
# Get data information
print(len(train_x),len(test_x))

120 30


In [5]:
# Feature columns describe what is the input look like.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [6]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa94d0b5bd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpiyuRny', '_save_summary_steps': 100}


In [7]:
# Train the Model.
classifier.train(input_fn=lambda:iris_data.train_input_fn(train_x, train_y,args.batch_size),\
                 steps=args.train_steps)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpiyuRny/model.ckpt.
INFO:tensorflow:loss = 160.43016, step = 1
INFO:tensorflow:global_step/sec: 397.104
INFO:tensorflow:loss = 34.871452, step = 101 (0.253 sec)
INFO:tensorflow:global_step/sec: 437.38
INFO:tensorflow:loss = 15.658861, step = 201 (0.229 sec)
INFO:tensorflow:global_step/sec: 444.543
INFO:tensorflow:loss = 9.891801, step = 301 (0.225 sec)
INFO:tensorflow:global_step/sec: 349.859
INFO:tensorflow:loss = 8.247163, step = 401 (0.286 sec)
INFO:tensorflow:global_step/sec: 329.635
INFO:tensorflow:loss = 6.6800284, step = 501 (0.303 sec)
INFO:tensorflow:global_step/sec: 433.445
INFO:tensorflow:loss = 7.781276, step = 601 (0.231 sec)
INFO:tensorflow:global_step/sec: 440.512
INFO:tensorflow:loss = 6.563202, step = 701 (0.227 sec)
INFO:tensorflow:global_step/sec: 298.061
INFO:tensorflow:loss = 6.7254314, step = 801 (0.337 sec)
INFO:tensorflow:global_step/sec: 321.361
INFO:tensorflow:loss

In [8]:
# Evaluate the model.
eval_result = classifier.evaluate(
input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,args.batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-02-12-13:53:42
INFO:tensorflow:Restoring parameters from /tmp/tmpiyuRny/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-12-13:53:42
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.06982872, global_step = 1000, loss = 2.0948615

Test set accuracy: 0.967



In [10]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(input_fn=lambda:iris_data.eval_input_fn(predict_x,\
                                        labels=None, \
                                        batch_size=args.batch_size))

In [11]:
for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],100 * probability, expec))

INFO:tensorflow:Restoring parameters from /tmp/tmpiyuRny/model.ckpt-1000

Prediction is "Setosa" (99.2%), expected "Setosa"

Prediction is "Versicolor" (98.7%), expected "Versicolor"

Prediction is "Virginica" (98.7%), expected "Virginica"


In [12]:
#  Testing on the training data
# Convert the inputs to a Dataset.
# train_x, train_y, batch_size = 100
# https://www.tensorflow.org/api_docs/python/tf/contrib/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))

In [34]:
print(dataset)

<TensorSliceDataset shapes: ({SepalLength: (), PetalWidth: (), PetalLength: (), SepalWidth: ()}, ()), types: ({SepalLength: tf.float64, PetalWidth: tf.float64, PetalLength: tf.float64, SepalWidth: tf.float64}, tf.int64)>


In [ ]:
# https://www.tensorflow.org/get_started/datasets_quickstart

In [27]:
my_feature_columns

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [28]:
matrix_feature_column = tf.feature_column.numeric_column(key="MyMatrix",
                                                         shape=[10,5])
matrix_feature_column

_NumericColumn(key='MyMatrix', shape=(10, 5), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [33]:
type(eval_result)

dict

In [35]:
type(my_feature_columns)

list